# Beer Reviews 

This example analyzes beer reviews to find the most common words used in positive and negative reviews.
Original example can be found [here](https://medium.com/rapids-ai/real-data-has-strings-now-so-do-gpus-994497d55f8e)

### Notes on running these queries:

By defaults runs use Bodo. Hence, data is distributed in chunks across processes.

reviews_sample.csv size is 23.1MB

Fulldataset is available on "s3://bodo-examples-data/beer/reviews.csv" and its size is 2.2GB

To run the code:
1. Make sure you [add your AWS account credentials to Saturn Cloud](https://saturncloud.io/docs/examples/python/load-data/qs-load-data-s3/#create-aws-credentials) to access the data.
2. If you want to run the example using pandas only (without Bodo):
    1. Comment lines magic expression (`%%px`) and bodo decorator (`@bodo.jit`) from all the code cells.
    2. Then, re-run cells from the beginning.

### Start an IPyParallel cluster
Run the following code in a cell to start an IPyParallel cluster. 4 cores are used in this example. 

In [ ]:
import ipyparallel as ipp
import psutil

n = min(psutil.cpu_count(logical=False), 8)
rc = ipp.Cluster(engines="mpi", n=n).start_and_connect_sync(activate=True)

### Verifying your setup
Run the following code to verify that your IPyParallel cluster is set up correctly:

In [ ]:
%%px
import bodo

print(f"Hello World from rank {bodo.get_rank()}. Total ranks={bodo.get_size()}")

## Importing the Packages

These are the main packages we are going to work with:
 - Bodo to parallelize Python code automatically
 - Pandas to work with data

In [ ]:
%%px
import warnings

warnings.filterwarnings("ignore")

import time

import bodo
import pandas as pd

## Preprocessing
1. Create lists of stopwords and punctuation that will be removed.
2. Define regex that will be used to remove these punctuation and stopwords from the reviews.
3. Use the lower and strip functions to convert all letters to lowercase and remove excess whitespace. 
4. Remove stopwords and punctuation. 

In [ ]:
%%px
with open("nltk-stopwords.txt", "r") as fh:
    STOPWORDS = list(map(str.strip, fh.readlines()))


PUNCT_LIST = ["\.", "\-", "\?", "\:", ":", "!", "&", "'", ","]
punc_regex = "|".join([f"({p})" for p in PUNCT_LIST])
stopword_regex = "|".join([f"\\b({s})\\b" for s in STOPWORDS])

In [ ]:
%%px
@bodo.jit(distributed=["reviews"])
def preprocess(reviews):
    # lowercase and strip
    reviews = reviews.str.lower()
    reviews = reviews.str.strip()

    # remove punctuation and stopwords
    reviews = reviews.str.replace(punc_regex, "", regex=True)
    reviews = reviews.str.replace(stopword_regex, "", regex=True)
    return reviews

## Find the Most Common Words

In [ ]:
%%px
@bodo.jit
def find_top_words(review_filename):
    # Load in the data
    t_start = time.time()
    df = pd.read_csv(review_filename, parse_dates=[2])
    print("read time", time.time() - t_start)

    score = df.score
    reviews = df.text

    t1 = time.time()
    reviews = preprocess(reviews)
    print("preprocess time", time.time() - t1)

    t1 = time.time()
    # create low and high score series
    low_threshold = 1.5
    high_threshold = 4.95
    high_reviews = reviews[score > high_threshold]
    low_reviews = reviews[score <= low_threshold]
    high_reviews = high_reviews.dropna()
    low_reviews = low_reviews.dropna()

    high_colsplit = high_reviews.str.split()
    low_colsplit = low_reviews.str.split()
    print("high/low time", time.time() - t1)

    t1 = time.time()
    high_words = high_colsplit.explode()
    low_words = low_colsplit.explode()

    top_words = high_words.value_counts().head(25)
    low_words = low_words.value_counts().head(25)
    print("value_counts time", time.time() - t1)
    print("total time", time.time() - t_start)

    print(top_words)
    print(low_words)


find_top_words("s3://bodo-examples-data/beer/reviews_sample.csv")

In [ ]:
# To stop the cluster run the following command.
rc.cluster.stop_cluster_sync()